# Azure Quantum Optimization Sample: Secret Santa

This sample walks through how to solve the Secret Santa problem using Azure Quantum. The scenario is defined as follows:

- Vincent, Tess, and Uma each write their name on a slip of paper and place the paper in a jar.
- Everybody then draws a slip of paper from the jar at random.
- Each person buys a small gift and writes a poem for the person whose name they have drawn.
  - If they draw their own name, they return the slip of paper and re-draw.

> **Note:**
> The inspiration for this scenario came from Vincent's blog post ([found here](https://vincent.frl/quantum-secret-santa/)), which demonstrates how to use [Q# and the QDK](https://docs.microsoft.com/azure/quantum/overview-what-is-qsharp-and-qdk) to solve this scenario. In this sample, we will make use of the [Azure Quantum QIO service](https://docs.microsoft.com/azure/quantum/optimization-what-is-quantum-optimization) to solve the same problem.

## Introduction: binary optimization

Binary optimization problems take the general form:

$\text{Minimize: } \sum_{k} term_k = \sum_k c_k \prod_{i} x_i \text{ where } x_i \in \{ 0,1 \} \text{ or } \{ -1 , 1 \} \text{ and } c_k \in \mathbb{R} $

Our job is to define a mathematical representation of our problem in this binary format and then use Azure Quantum to solve it.

For example, the problem shown below:

$13 + 17x_0 + 23x_1x_3x_{77},$

would be represented by the following Terms in the Azure Quantum Python SDK:

```python
terms = [Term(c = 13.0, indices = []), Term(c=17.0, indices = [0]) , Term(c = 23.0, indices = 1, 3, 77)] 
```

> **Note:** See [this documentation page](https://docs.microsoft.com/azure/quantum/quickstart-microsoft-qio?pivots=platform-microsoft#express-a-simple-problem) for further information.

## Binary formulation for the Secret Santa problem

To represent the Secret Santa problem, we can use six binary variables, as outlined in the Scenario Table below:

|- buys ->|**Vincent**|**Tess**|**Uma**|
|--|--|--|--|
|**Vincent**|--|$x_0$|$x_1$|
|**Tess**|$x_2$|--|$x_3$|
|**Uma**|$x_3$|$x_4$|--|

The constraints for the problem can be expressed as doing logical ANDs ($ \land $) of variables that are EXCLUSIVE-ORd ($ \oplus $) together, like this:

$
( x_0 \oplus x_1 ) \land ( x_2 \oplus x_3 ) \land ( x_4 \oplus x_5 ) \land ( x_2 \oplus x_4 ) \land ( x_0 \oplus x_5 ) \land ( x_1 \oplus x_3 )
$

$
\text{ where } x_i \in \{ 0,1 \} 
$


The truth table for exclusive or ($ \oplus $) is shown below (one variable or the other is **one**, but not both):

|$x_0$|$x_1$|$x_0 \oplus x_1$|
|--|--|--|
|0|0|0|
|0|1|1|
|1|0|1|
|1|1|0|

Using this truth table, we can see how the constraints are derived. Looking at the Scenario Table defined previously:

- Reading the first **row** of the table, Vincent may buy a gift and write a poem for Tess or for Uma, but not both.
- Reading the first **column** of the table, Vincent may receive a gift and poem from Tess or from Uma, but not both.

More generally:

- Each person should give and receive **exactly one** gift from one other person in the group.
  - If a person gives more or less than one gift, or receives more or less than one gift, this constraint has been violated and the solution will not be valid.

So, how do we represent $ ( x_0 \oplus x_1 ) $ in a binary format that Azure Quantum will understand?

Keeping in mind that we want to **minimize** our cost function, let's try to use the following representation:

$ ( x_0 + x_1 - 1 )^2 $  

Let's check the truth table for this formulation:

|$x_0$|$x_1$|$(x_0 + x_1 - 1)^2$|
|--|--|--|
|0|0|1|
|0|1|0|
|1|0|0|
|1|1|1|

As you can see, in rows where there is exactly one $1$, the result is $0$. This means the penalty applied in those situations will be $0$. Since we want to minimize the cost function, getting $0$ for the answers we want is the correct result.

We are almost there! The next step is to do a [quadratic expansion of this formula](https://en.wikipedia.org/wiki/Polynomial). This leaves us with the following expanded formula:

$ x_0^2 + x_1^2 + 2x_0x_1 - 2x_0 - 2x_1 + 1 $

We build up the Terms in the helper function `build_terms` shown below, but instead of using $x_0$ and $x_1$, we use the indices for our variables instead ($i$ and $j$). 

So for example, $x_0 \oplus x_1$ (where $i = 0$ and $j = 1$) would translate to `build_terms(0, 1)`.

In [ ]:
from azure.quantum import Workspace
workspace = Workspace (
    subscription_id = "421b563f-a977-42aa-8934-f41ca5664b73",
    resource_group = "azurequantum",
    name = "qsamples",
    location = "westus"
)

In [ ]:
# Import required modules
from typing import List
from azure.quantum.optimization import Problem, ProblemType, Term, SimulatedAnnealing 

def build_terms(i: int, j: int):
    """
    Construct Terms for a row or a column (two variables) of the Secret Santa matrix

    Arguments:
    i (int): index of first variable
    j (int): index of second variable

    """
    
    terms = []                                      # Initialize empty terms list
    terms.append(Term(c = 1.0, indices = [i, i]))   # x(i)^2
    terms.append(Term(c = 1.0, indices = [j, j]))   # x(j)^2
    terms.append(Term(c = 2.0, indices = [i, j]))   # 2x(i)x(j) 
    terms.append(Term(c = -2.0, indices = [i]))     # -2x(i)
    terms.append(Term(c = -2.0, indices = [j]))     # -2x(j)
    terms.append(Term(c = 1.0, indices = []))       # +1

    return terms

We have one more helper function, which takes the answer returned from the service and interprets it in a human-readable way based on the Scenario Table, above.

In [ ]:
def print_results(config: dict
):
    """
    print results of run

    Arguements:
    config (dictionary): config returned from solver
    """
    result = {
                '0': 'Vincent buys Tess a gift and writes her a poem.',
                '1': 'Vincent buys Uma a gift and writes her a poem.',
                '2': 'Tess buys Vincent a gift and writes him a poem.',
                '3': 'Tess buys Uma a gift and writes her a poem.',
                '4': 'Uma buys Vincent a gift and writes him a poem.',
                '5': 'Uma buys Tess a gift and writes her a poem.'}

    for key, val in config.items():
        if val == 1:
            print(result[key])

Bringing it all together:

In [ ]:
"""
build secret santa matrix

        Vincent Tess Uma
Vincent    -    x(0) x(1)
Tess      x(2)   -   x(3)
Uma	      x(4)  x(5)  -
"""

#       row 0             + row 1             + row 2                
terms = build_terms(0, 1) + build_terms(2, 3) + build_terms(4, 5)

#             + col 0             + col 1             + col 2
terms = terms + build_terms(2, 4) + build_terms(0, 5) + build_terms(1, 3)

print(f'Terms: {terms}\n')

problem = Problem(name = 'secret santa', problem_type = ProblemType.pubo, terms = terms)

solver = SimulatedAnnealing(workspace, timeout = 2)

print('Submitting problem to Azure Quantum')
result = solver.optimize(problem)

print(f'\n\nResult: {result}\n')

In [ ]:
# Display the final human-readable result
print('Human-readable solution:')
print_results(result['configuration'])